# Y Combinator - Recursion
Having developed numerals, Boolean algebra and arithmetic for $\lambda$-Calculus, the only element remaining to make it Turing complete is infinite loops. We can achieve this through recursion.

First, let's define what we had previously through our other notebook on Church encoding, namely numerals, Boolean algebra, arithmetic and a conditional statement to check if zero.

In [23]:
#Booleans
T = lambda t: lambda f: t
F = lambda t: lambda f: f

#use annotations to tell what function is returned in verifying the result
T.__annotations__['T'] = True
F.__annotations__['F'] = False

#also can tell by converting to Python bool type, cheating a bit though
to_bool = lambda boolean: boolean(True)(False)

#Boolean algebra
NOT = lambda boolean: boolean(F)(T)
AND = lambda bool1: lambda bool2: bool1(bool2)(F)
OR = lambda bool1: lambda bool2: bool1(bool1)(bool2)

In [24]:
#Numerals
#identity
I = lambda x: x

#numerals
N0 = lambda f: lambda x: x
N1 = lambda f: lambda x: f(x)
N2 = lambda f: lambda x: f(f(x))
N3 = lambda f: lambda x: f(f(f(x)))
N4 = lambda f: lambda x: f(f(f(f(x))))

#use annotations to tell what function is returned in verifying the result
I.__annotations__['I'] = "I"
N0.__annotations__['0'] = 0
N1.__annotations__['1'] = 1
N2.__annotations__['2'] = 2
N3.__annotations__['3'] = 3
N4.__annotations__['4'] = 4

#also by converting to Python int type by counting function calls
#we pass the increment function into the numeral to count the calls
to_int = lambda n: n(lambda integer: integer + 1)(0)

In [25]:
#Arithmetic
#Count
S = lambda n: lambda f: lambda x: f(n(f)(x))
ADDS = lambda m: lambda n: lambda f: lambda x: m(S)(n)(f)(x)
MUL = lambda m: lambda n: lambda f: lambda x: m(n(f))(x)
POW = lambda m: lambda n: lambda f: lambda x: S(n)(m(f))(x)

In [26]:
#check for zero
Z = lambda x: x(lambda x: F)(T)

## Combinators

Just as for Boolean algebra and linear algebra, where we define a minimal set of operators that can replicate all of the operations necessary in that system, we can define a non-redundant or minimal set of combinators, i.e. $\lambda$ expressions without free variables that combine arguments in different ways. 

In this way, we can use combinators as the atoms of $\lambda$-Calculus and problem can be solved or algorithms constructed like molecules from these atomic structures just as atoms from the periodic table in chemistry. This modular approach was pioneered by Haskell Curry and he provided distinct names for these expressions and reformulated $\lambda$-Calculus based on them.

More information can be found in the $\lambda$-Calculus chapter of the book.

## Self Application

The surprising result is that in a system where only unary, pure functions exist, we can still create the notion of recursion using the Mockingbird combinator $\mathcal{M}$ as a starting point

$$\mathcal{M} := \lambda f.\ ff$$
We can actually redefine OR to reuse the Mockingbird, noting the slightly alternative definition above from the previous notebook.

In [27]:
M = lambda f: f(f)
ORM = lambda bool1: lambda bool2: M(bool1)(bool2)

In [31]:
OR(F)(T).__annotations__

{'T': True}

In [32]:
ORM(F)(T).__annotations__

{'T': True}

We can see the initial signs of recursion (and indeed the halting problem) when we try to self apply the Mockingbird to itself

In [35]:
try:
    M(M)
except RuntimeError as err:
    print(err)

maximum recursion depth exceeded


But we need something more useful including passing our own function and handling stopping conditions to prevent stack overflows. What we need is a recursive combinator that takes in a function $f$, but calls $f$ again with the recursive combinator only if the base condition is not satisfied. If the base condition is satisfied, we would like to stop calling it itself and allow $\beta$ reduction. This is exactly what the $\mathcal{Y}$ combinator allows us to do and is defined as
$$
\mathcal{Y}   := \lambda f.\ (\lambda x.\ f(xx))(\lambda x.\ f(xx))
$$
which can also be written in terms of the Mockingbird
$$
\mathcal{Y}   := \lambda f.\ \mathcal{M}(\lambda x.\ f(\mathcal{M}x))
$$
We can see how this might work from a top level $\beta$ reduction involving some combinator $\mathcal{R}$ as
\begin{align}
 \mathcal{Y}\mathcal{R}   &= (\lambda f.\ (\lambda x.\ f(xx))(\lambda x.\ f(xx)))\mathcal{R}\\
                    &= (\lambda x.\ \mathcal{R}(xx))(\lambda x.\ \mathcal{R}(xx))\nonumber\\
                    &= \mathcal{R}((\lambda x.\ \mathcal{R}(xx))(\lambda x.\ \mathcal{R}(xx)))\nonumber\\
                    &= \mathcal{R}(\mathcal{Y}\mathcal{R})
\end{align}
To demonstrate this amazing structure in a practical sense, we will show how to build the factorial function $n!$ for the number $n$. Consider the recursive form of the factorial function
\begin{align}
 n! = 
    \begin{cases}
    1,          &\textrm{if } n=0\\
    n(n-1)!,    &\textrm{for } n>0.
    \end{cases}
\end{align}
In standard procedural Python we might write it as

In [37]:
def fac(n):
    '''
    Recursive factorial function
    '''
    if n == 0:
        return 1
    return n*fac(n-1)

In [38]:
fac(5)

120

We can attempt to convert the above procedural function directly to $\lambda$-Calculus for which we might (incorrectly) write
\begin{equation}
 \textrm{FAC} = \lambda n.\ \textrm{if } \textrm{Z}(n)(1)\ \textrm{MUL}(n)(\textrm{FAC}(\textrm{PRED}(n)))
\end{equation}
Instead, because functions in $\lambda$-Calculus are anonymous, we can write the above correctly using the $\mathcal{Y}$ combinator as
\begin{eqnarray}
 \textrm{FAC} = \mathcal{Y}( \lambda f.\ \lambda n.\ \textrm{Z}(n)(1)\ \textrm{MUL}(n)(f(\textrm{PRED}(n))))
\end{eqnarray}
